In [4]:
# import pandas as pd

In [5]:
# from fuzzywuzzy import fuzz as sim

In [5]:
# x = sim.token_set_ratio('Room, 2 Double Beds (19th to 25th Floors)', 'Two Double Beds - Location Room (19th to 25th Floors)')
# x

In [97]:
# import pandas as pd
# from fuzzywuzzy import fuzz as sim
# def prediksi(gejala, namafile):
# #     print(namafile)
#     data = pd.read_excel(namafile)
#     gejala_list = data["Tanda_dan_Gejala"].tolist()
#     diagnosa_list = data["Nama Penyakit"].tolist()
#     detail = data["Detail Nama Penyakit"].tolist()
# #     cek = gejala
# #     data_latih = ["a", "b"]
# #     diagnosa = ['aa','bb']
#     ratio=list()
#     for i in gejala_list:
#         rat = fuzz.token_set_ratio(i, gejala)
#         ratio.append(rat)

#     dict_ = {
#         "similarity":ratio,
#         "diagnosa":diagnosa_list,
#         "detail":detail
#     }


#     dataf = pd.DataFrame.from_dict(dict_)
# #     print(dataf)
# #     sorted_ = dataf.sort_values(by=['similarity'], descending=True)
#     sorted_ =  dataf.sort_values(by=['similarity'], ascending=False)
# #     print(sorted_)
#     diagnosa = sorted_['diagnosa'].tolist()[0]
#     similarity = sorted_['similarity'].tolist()[0]
#     detail = sorted_['detail'].tolist()[0]
    
#     return [diagnosa, similarity, detail, sorted_]


In [2]:
#data_train_infection
# namafile = "data_train_infection.xlsx"


# gejala = "Nausea dan vomitus. Penurunan selera makan. Demam. Berkembang hingga menimbulkan nyeri abdomen, muntah darah, dan diare hebat."
# print("Tanda dan Gejala :",gejala)
# print("similarity       :",prediksi(gejala, namafile)[1])
# print("Nama Penyakit    :",prediksi(gejala, namafile)[0])
# print("Detail  Penyakit :",prediksi(gejala, namafile)[2])


In [26]:
import pandas as pd
import json
from tokenisasi import  tokenisasi as ts
import json
from joblib import dump
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
data = pd.read_excel("data_train_infection.xlsx")
gejala_list = data["Tanda_dan_Gejala"].tolist()
diagnosa_list = data["Nama Penyakit"].tolist()
detail = data["Detail Nama Penyakit"].tolist()
Jenis_Infeksi = data["Jenis Infeksi"].tolist()

dict_ = {
    "gejala":gejala_list,
    "nama_penyakit":diagnosa_list,
    "detail":detail,
    "Jenis_Infeksi":Jenis_Infeksi
}
with open("prediksi\model\penyakit.json","w")as f:
    json.dump(dict_, f)
# data

In [38]:
import json
from joblib import dump
from sklearn.feature_extraction.text import TfidfVectorizer
# corpus = gejala_list
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus)
# save = vectorizer.fit(corpus)
# # dump(save, 'prediksi/model/tfidf_model.sav')
# # print(vectorizer.get_feature_names())
# X = X.toarray()
# # print(X.shape)
# X = X.tolist()
# # with open("prediksi/model/vector_model.json","w")as f:
# #     json.dump(X, f)

In [18]:
# X = X.tolist()
# with open("prediksi/model/vector_model.json","w")as f:
#     json.dump(X, f)

In [43]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
n = list()
for i in gejala_list:
    t_kata = tknzr.tokenize(i)
    for j in t_kata:
        n.append(j)

In [28]:
# for i in list(set(n)):
#     print(i)

In [51]:
deli = """
di
oleh
atau
dan
"
/
)
awalnya
(
-
ada
ini
lagi
,
itu
)
:
.
dengan
disertai
dari
karena
yang
pada
serta
dapat
untuk
hingga
"""

for i in deli.split():
    print(i, end="|")
deli = deli.split()
with open("prediksi/model/delimiter.json", "w") as f:
    json.dump(deli, f)
new_gejala = list()
for i in gejala_list:
    new_gejala.append((ts.ngramku(i, n=10, delimiter = deli)['string'].lower()))
    
corpus = new_gejala
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
save = vectorizer.fit(corpus)
dump(save, 'prediksi/model/tfidf_model.sav')
# print(vectorizer.get_feature_names())
X_ = X.toarray()
# print(X.shape)
X = X_.tolist()
with open("prediksi/model/vector_model.json","w")as f:
    json.dump(X, f)

di|oleh|atau|dan|"|/|)|awalnya|(|-|ada|ini|lagi|,|itu|)|:|.|dengan|disertai|dari|karena|yang|pada|serta|dapat|untuk|hingga|

In [87]:
# samples = X_
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_, diagnosa_list)
dump(neigh, 'prediksi/model/knn_model_diagnosa.sav')

neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_, detail)
dump(neigh, 'prediksi/model/knn_model_detail.sav')

print(dump)

<function dump at 0x000001C14B5D42F0>


In [88]:
x  = neigh.predict([X[0]])#predict([X[0]])
x[0]

'Kutaneus'

In [86]:
x[0]

'Antrax'